# 前情
HW7的任務是模型壓縮 - Neural Network Compression

Compression有很多種門派，在這裡我們會介紹上課出現過的其中四種，分別是:
* 知識蒸餾 Knowledge Distillation
* 網路剪枝 Network Pruning
* 用少量參數來做 CNN Architecture Design
* 參數量化 Weight Quantization

在這個 notebook 中我們會介紹 Network Pruning，而我們有提供已經做完 Knowledge Distillation 的小 model 來做 Pruning。

* Model架構 / Architecute Design在同目錄中的hw7_Architecture_Design.ipynb。
  * 參數為 base=16, width_mult=1 (default)

本次的 model 要求必须将图片直接读入，否则准确率下降到老师例子的一半，非常耗费内存，只能减少训练 dataset 到原来的 10%，但由于是在原有model基础上继续训练，精度下降非常小

In [1]:
#coding=utf8

import torch
import os
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models

# Network Pruning
将一個已經學完的 model 中的 neuron 進行刪減，讓整個網路變得更瘦。
<img src="hw7_data/Neuron Pruning.png" width="500px">

## Weight & Neuron Pruning
- **neuron pruning**： prune 掉一個 neuron 就等於是把一個 matrix 的整個 column 全部砍掉，matrix 整體變小，运行速度就會比較快
- **weight pruning**： matrix大小不變，只是有很多 0

## What to Prune?
- 先衡量 Neuron 的重要性，衡量完所有的 Neuron 後，就可以把比較不重要的 Neuron 刪減掉。
- 在這裡我們介紹一個很簡單可以衡量 Neuron 重要性的方法 - 就是看batchnorm layer的$\gamma$因子來決定neuron的重要性。 (by paper - Network Slimming)
 - $\bar X$为均值，Var(x)为方差, eps 防止分母为 0， γ 和β是学习得到的
$$ y = {{x-\bar X} \over {\sqrt {Var(X)+ eps}}} \times \gamma + bias $$
 - 相信大家看這個pytorch提供的batchnorm公式應該就可以意識到為甚麼$\gamma$可以當作重要性來衡量了:)

- Network Slimming其實步驟沒有這麼簡單，有興趣的同學可以check以下連結。[Netowrk Slimming](https://arxiv.org/abs/1708.06519)

## 為甚麼這會 work?
- 樹多必有枯枝，有些 neuron 只是在躺分，所以有他沒他沒有差
- 困難的說可以回想起老師說過的大樂透假說(The Lottery Ticket Hypothesis)就可以知道了

## 要怎麼實作?
- 為了避免複雜的操作，我們會將 StudentNet(width_mult = $\alpha$)的 neuron 經過篩選後移植到 StudentNet(width_mult=$\beta$)。($\alpha  >  \beta$)
- 篩選的方法也很簡單，只需要抓出每一個 block的 batchnorm 的 $\gamma$ 即可。

## 一些實作細節
- 假設model中間兩層是這樣的:

|Layer|Output # of Channels|
|-|-|
|Input|in_chs|
|Depthwise(in_chs)|in_chs|
|BatchNorm(in_chs)|in_chs|
|Pointwise(in_chs, **mid_chs**)|**mid_chs**|
|**Depthwise(mid_chs)**|**mid_chs**|
|**BatchNorm(mid_chs)**|**mid_chs**|
|Pointwise(**mid_chs**, out_chs)|out_chs|

則你會發現利用第二個 BatchNorm 來做篩選的時候，跟他的 Neuron 有直接關係的是該層的 Depthwise & Pointwise 以及上層的 Pointwise。
因此再做 neuron 篩選時記得要將這四個(包括自己, bn)也要同時 prune 掉。

- 在 Design Architecure 內，model 的一個 block，名稱所對應的 Weight；

|#|name|meaning|code|weight shape|
|-|-|-|-|-|
|0|cnn.{i}.0|Depthwise Convolution Layer|nn.Conv2d(x, x, 3, 1, 1, group=x)|(x, 1, 3, 3)|
|1|cnn.{i}.1|Batch Normalization|nn.BatchNorm2d(x)|(x)|
|2||ReLU6|nn.ReLU6||
|3|cnn.{i}.3|Pointwise Convolution Layer|nn.Conv2d(x, y, 1),|(y, x, 1, 1)|
|4||MaxPooling|nn.MaxPool2d(2, 2, 0)||

In [2]:
def network_slimming(old_model, new_model):
    params = old_model.state_dict()
    new_params = new_model.state_dict()
    selected_idx = [] # selected_idx: 每一層所選擇的 neuron 序号
    
    for i in range(8): # 我們總共有7層CNN，因此逐一抓取選擇的 neuron index 們。
        importance = params[f'cnn.{i}.1.weight'] # 根據上表，Batch Normalization 层表示参数的重要性，要抓的 gamma 係數在 cnn.{i}.1.weight 內
        old_dim = len(importance) # Batch Normalization 层總共有幾個 neuron
        new_dim = len(new_params[f'cnn.{i}.1.weight']) # 每次剪枝的比例不同，一层總共有幾個 neuron
        ranking = torch.argsort(importance, descending = True) # 以 Ranking 做 Index 排序，較大的在前面
        selected_idx.append(ranking[ : new_dim]) # 把篩選結果放入selected_idx中

    now_processed = 1 # 记录处理到哪一层，第 0 层不prune，从第 1 层开始处理
    for (name, p1), (name2, p2) in zip(params.items(), new_params.items()):
        if name.startswith('cnn') and p1.size() != torch.Size([]) and now_processed != len(selected_idx): # 如果是cnn層 & 參數 > 0個數字 & 不是最后一层則移植參數
            if name.startswith(f'cnn.{now_processed}.3'): # 每层的第三步都是 Pointwise，表示該層的移植已經完成，讓 now_processed + 1
                now_processed += 1
            if name.endswith('3.weight'): # pointwise 的 weight 會被上一層的 pruning 和下一層的 pruning 所影響，因此需要特判
                if len(selected_idx) == now_processed: # 如果是最後一層cnn，則輸出的 neuron 不需要prune掉
                    new_params[name] = p1[:, selected_idx[now_processed - 1]] # selected_idx[now_processed - 1] 是一个filter
                else: # 反之，就依照上層和下層所選擇的index進行移植
                    new_params[name] = p1[selected_idx[now_processed]][:, selected_idx[now_processed - 1]] # Conv2d(x,y,1)的weight shape是(y,x,1,1)，順序是反的
            else: # 如果不是 pointwise，直接複製
                new_params[name] = p1[selected_idx[now_processed]]
        else: # 如果是FC層，或是該參數只有一個數字(例如 batchnorm 的 tracenum 等等資訊)，那麼就直接複製
            new_params[name] = p1

    # 讓新model load進被我們篩選過的parameters，並回傳new_model。        
    new_model.load_state_dict(new_params)
    return new_model

# Data Processing

我們的 Dataset 使用的是跟 hw3 - CNN 同樣的 Dataset

## Read image 
本次的 model 必须将图片直接读入，否则准确率下降到老师例子的一半，非常耗费内存，只能减少训练规模

In [3]:
import time
import numpy as np
from PIL import Image

def readfile(path, label):# label 是0 或 1，1代表需要回傳 y 值
    image_dir = sorted(os.listdir(path))
    # uint8是专门用于存储各种图像的，范围是从0–255
    x = [] #初始化，
    y = np.zeros((len(image_dir)), dtype=np.uint8) #初始化
    # 给文件夹里的图片一个编号，并将编号和图片组合成一个表
    for i, file in enumerate(image_dir):
        image = Image.open(path + "/" + file)
        image_fp = image.fp # Get File Descriptor
        image.load()
        image_fp.close() # Close File Descriptor (or it'll reach OPEN_MAX)
        x.append(image)
        if label:
            # 训练集图像命名方式为 [类别]_[第几张图片].jpg
            y[i] = int(file.split("_")[0]) #图片名分成2个部分，取前面的一个，得到类别作为 y 值
    if label:
      return x, y
    else:
      return x

# 分別將 training set、validation set、testing set 用 readfile 函式讀進來
workspace_dir = 'hw7_data/'
start_time = time.time()
print("Reading data")
train_x, train_y = readfile(os.path.join(workspace_dir, "training"), True)
print("Size of training data = {}".format(len(train_x)))
val_x, val_y = readfile(os.path.join(workspace_dir, "validation"), True)
print("Size of validation data = {}".format(len(val_x)))
test_x = readfile(os.path.join(workspace_dir, "testing"), False)
print("Size of Testing data = {}".format(len(test_x)))
end_time = time.time()
print("用时：", end_time - start_time)

Reading data
Size of training data = 1000
Size of validation data = 500
Size of Testing data = 124
用时： 6.789473056793213


## 预处理

In [4]:
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset

# 来自torchvision.transforms
# 训练数据做数据增强 (data augmentation)
train_transform = transforms.Compose([ # 处理图片, 用Compose把多个处理步骤整合到一起
    #transforms.ToPILImage(), #把数据转换为tensfroms格式
    transforms.RandomCrop(256, pad_if_needed = True, padding_mode='symmetric'), # 随机裁剪到256*256
    transforms.RandomHorizontalFlip(), # 隨機將圖片左右镜像
    transforms.RandomRotation(15), # 隨機旋轉圖片
    transforms.ToTensor(), # 將圖片轉成 Tensor，並把數值 normalize 到 [0,1] ,这个格式可以直接输入进神经网络了
])

# 测试数据不需做 数据增强 (data augmentation)
test_transform = transforms.Compose([
    #transforms.ToPILImage(),   
    transforms.CenterCrop(256),
    transforms.ToTensor(),
])

# 定义ImgDataset类，继承torch.utils.data.Dataset，实现数据读取方式
# torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False,...) 
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        self.y = y
        if y is not None: #将 y 变成一个长向量 64-bit integer (signed)
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index): #取一张图片和对应的分类 y 值
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None: # 判断是否需要 y 值
            Y = self.y[index]
            return X, Y
        else:
            return X

train_set = ImgDataset(train_x, train_y, train_transform) #训练集
val_set = ImgDataset(val_x, val_y, test_transform) #验证集

## 载入数据

In [5]:
from torch.utils.data import DataLoader

batch_size = 32

# 使用torch.utils.data.DataLoader(), 实现数据的批量读取
#dataset：加载的数据集(Dataset对象); batch_size：batch size; shuffle:：是否将数据打乱
train_loader = DataLoader(train_set, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size = batch_size, shuffle=False)

# 载入模型
载入提供的模型，设置模型参数

In [6]:
from hw7_data import hw7_Architecture_Design as Architecture_Design

net = Architecture_Design.StudentNet()
net.load_state_dict(torch.load('hw7_data/student_custom_small.bin'))

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# Start Training

- 每次Prune rate是0.95(每层删去 5% 的filter)，Prune 完後會重新 fine-tune 3 epochs。
- 其餘的步驟與你在做hw3 - CNN的時候一樣。

In [7]:
def train_epoch(train_loader):
    train_acc, train_loss = 0, 0
    for i, data in enumerate(train_loader): # 给train_loader的矩阵一个编号，并组合成一个表
        optimizer.zero_grad() # 將 model 參數的 gradient 歸零
        train_pred = net(data[0].cuda()) # 利用 model 算出来的機率分佈 這邊實際上就是去呼叫 model 的 forward 函數
        batch_loss = loss(train_pred, data[1].cuda()) # 計算 loss （注意 prediction 跟 label 必須同時在 CPU 或是 GPU 上）
        
        batch_loss.backward() # 利用 back propagation 算出每個參數的 gradient
        optimizer.step() # 以 optimizer 用 gradient 更新參數值

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()
    return train_loss/train_set.__len__(), train_acc/train_set.__len__()

def val_epoch(val_loader):
    val_acc, val_loss = 0 ,0
    with torch.no_grad(): #torch.no_grad() 是一个上下文管理器，被该语句 wrap 起来的部分将不会track 梯度
        for i, data in enumerate(val_loader):
            val_pred = net(data[0].cuda())
            batch_loss = loss(val_pred, data[1].cuda())

            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()
    return val_loss/val_set.__len__(), val_acc/val_set.__len__()

now_width_mult = 1
for i in range(5): # prune N 次
    now_width_mult *= 0.95 
    new_net = Architecture_Design.StudentNet(width_mult = now_width_mult).cuda()
    params = net.state_dict()
    net = network_slimming(net, new_net)
    now_best_acc = 0
    for epoch in range(5): #每 prune 一次训练 5 轮，选出最好的一轮
        epoch_start_time = time.time()
        net.train()
        train_loss, train_acc = train_epoch(train_loader)
        net.eval()
        valid_loss, valid_acc = val_epoch(val_loader)
        # 在每個width_mult的情況下，存下最好的model。
        if valid_acc > now_best_acc:
            now_best_acc = valid_acc
            torch.save(net.state_dict(), f'hw7_data/custom_small_rate_{now_width_mult}.bin')
        print('剩余 {:6.4f}% filter, 第{:>2d}轮 : [train loss: {:6.4f}, acc: {:6.4f}] [valid loss: {:6.4f}, acc: {:6.4f}], 用时：{:4.2f} sec(s)'.format(
            now_width_mult*100, epoch+1, train_loss, train_acc, valid_loss, valid_acc, time.time()-epoch_start_time))

剩余 95.0000% filter, 第 1轮 : [train loss: 0.0193, acc: 0.8490] [valid loss: 0.0426, acc: 0.7780], 用时：5.31 sec(s)
剩余 95.0000% filter, 第 2轮 : [train loss: 0.0179, acc: 0.8700] [valid loss: 0.0432, acc: 0.7760], 用时：4.31 sec(s)
剩余 95.0000% filter, 第 3轮 : [train loss: 0.0164, acc: 0.8630] [valid loss: 0.0429, acc: 0.7780], 用时：4.32 sec(s)
剩余 95.0000% filter, 第 4轮 : [train loss: 0.0182, acc: 0.8470] [valid loss: 0.0425, acc: 0.7680], 用时：4.36 sec(s)
剩余 95.0000% filter, 第 5轮 : [train loss: 0.0156, acc: 0.8650] [valid loss: 0.0422, acc: 0.7800], 用时：4.38 sec(s)
剩余 90.2500% filter, 第 1轮 : [train loss: 0.0215, acc: 0.8310] [valid loss: 0.0444, acc: 0.7700], 用时：4.57 sec(s)
剩余 90.2500% filter, 第 2轮 : [train loss: 0.0222, acc: 0.8270] [valid loss: 0.0468, acc: 0.7600], 用时：4.53 sec(s)
剩余 90.2500% filter, 第 3轮 : [train loss: 0.0219, acc: 0.8320] [valid loss: 0.0435, acc: 0.7720], 用时：4.54 sec(s)
剩余 90.2500% filter, 第 4轮 : [train loss: 0.0219, acc: 0.8320] [valid loss: 0.0434, acc: 0.7620], 用时：4.61 sec(s)
剩

# 测试

In [17]:
test_set = ImgDataset(test_x, transform=test_transform)
test_loader = DataLoader(test_set, batch_size = batch_size, shuffle=False)
name = ['面包', '奶', '甜品', '蛋', '油炸食品', '肉', '面条', '米饭', '海鲜', '汤', '果蔬']
prediction = []

new_net = Architecture_Design.StudentNet(width_mult = 0.95**5).cuda()
new_net.load_state_dict(torch.load('hw7_data/custom_small_rate_0.7737809374999999.bin'))
new_net.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = new_net(data.cuda())
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            prediction.append(y)
        
def rename_and_write_csv():
    f = os.listdir("hw7_data/testing_prediction")
    for i in range(len(f)):
        oldname = f[i]
        newname = oldname.split(".")[0] + '-'+ name[prediction[i]] + '.jpg'
        # 用os模块中的rename方法对文件改名
        os.rename("hw7_data/testing_prediction/" + oldname, "hw7_data/testing_prediction/" + newname)
    
    #將結果寫入 csv 檔
    with open("hw7_data/预测.csv", 'w') as f:
        f.write('Id,Category\n')
        for i, y in  enumerate(prediction):
            f.write('{},{},{}\n'.format(i, y, name[y]))
        
#rename_and_write_csv()